In [ ]:
import string

def generate_char_alphabet_and_translation(corpus_text):
  """
  Generiert ein zeichenbasiertes Alphabet und eine Übersetzungstabelle
  basierend auf einem gegebenen Textkorpus.

  Args:
    corpus_text (str): Der Textkorpus.

  Returns:
    tuple: Ein Tupel, das Folgendes enthält:
      - set: Das Alphabet (ein Set eindeutiger Zeichen).
      - dict: Die Übersetzungstabelle (Zeichen zu Index).
  """
  alphabet = set(corpus_text)

  char_to_index = {char: i for i, char in enumerate(sorted(list(alphabet)))}
  return alphabet, char_to_index

def generate_word_alphabet_and_translation(corpus_text):
  """
  Generiert ein wortbasiertes Alphabet und eine Übersetzungstabelle
  basierend auf einem gegebenen Textkorpus.

  Args:
    corpus_text (str): Der Textkorpus.

  Returns:
    tuple: Ein Tupel, das Folgendes enthält:
      - set: Das Alphabet (ein Set eindeutiger Wörter).
      - dict: Die Übersetzungstabelle (Wort zu Index).
  """

  # Einfache Worttokenisierung (kann durch komplexere Tokenizer ersetzt werden)
  words = corpus_text.lower().split()
  alphabet = set(words)

  word_to_index = {word: i for i, word in enumerate(sorted(list(alphabet)))}
  return alphabet, word_to_index

In [ ]:
generate_char_alphabet_and_translation("Hello World!!")

({' ', '!', 'H', 'W', 'd', 'e', 'l', 'o', 'r'},
 {' ': 0, '!': 1, 'H': 2, 'W': 3, 'd': 4, 'e': 5, 'l': 6, 'o': 7, 'r': 8})

In [ ]:
generate_word_alphabet_and_translation("Ich lerne gerne viel über die schöne Welt der KI")

({'der',
  'die',
  'gerne',
  'ich',
  'ki',
  'lerne',
  'schöne',
  'viel',
  'welt',
  'über'},
 {'der': 0,
  'die': 1,
  'gerne': 2,
  'ich': 3,
  'ki': 4,
  'lerne': 5,
  'schöne': 6,
  'viel': 7,
  'welt': 8,
  'über': 9})

In [ ]:
import requests
import tarfile
import os

url = "https://downloads.wortschatz-leipzig.de/corpora/deu_mixed-typical_2011_10K.tar.gz"
filename = "deu_mixed-typical_2011_10K.tar.gz"
extracted_dir = "deu_mixed-typical_2011_10K"
text_file_name = "deu_mixed-typical_2011_10K-sentences.txt"

# 1. Datei herunterladen
print(f"Lade Datei von {url} herunter...")
response = requests.get(url, stream=True)
if response.status_code == 200:
  with open(filename, 'wb') as f:
    f.write(response.raw.read())
  print("Download abgeschlossen.")
else:
  print(f"Fehler beim Herunterladen: Statuscode {response.status_code}")

# 2. Datei entpacken
print(f"Entpacke Datei {filename}...")
if os.path.exists(filename):
  with tarfile.open(filename, "r:gz") as tar:
    tar.extractall(".")
  print("Entpacken abgeschlossen.")
else:
  print(f"Fehler: Datei {filename} nicht gefunden.")

# 3. Text extrahieren
print(f"Extrahiere Text aus {text_file_name}...")
full_text = ""
file_path = os.path.join(extracted_dir, text_file_name)

if os.path.exists(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
      # Zeilenaufbau: Zeilennummer + Tab + Text
      parts = line.strip().split('\t', 1)
      if len(parts) > 1:
        full_text += parts[1] + " " # Füge den Text und ein Leerzeichen hinzu
  print("Textextraktion abgeschlossen.")
else:
  print(f"Fehler: Textdatei {file_path} nicht gefunden. Bitte prüfen Sie den Dateinamen im Tar-Archiv.")


Lade Datei von https://downloads.wortschatz-leipzig.de/corpora/deu_mixed-typical_2011_10K.tar.gz herunter...
Download abgeschlossen.
Entpacke Datei deu_mixed-typical_2011_10K.tar.gz...
Entpacken abgeschlossen.
Extrahiere Text aus deu_mixed-typical_2011_10K-sentences.txt...
Textextraktion abgeschlossen.


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers import pre_tokenizers
from tokenizers.trainers import BpeTrainer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

bpe_tokenizer = Tokenizer(BPE())

# Hier definieren wir, dass wir gerne mit dem Byte-Level Vor-Tokenisierer arbeiten
# würden. Dieser soll auch sicherstellen, dass jedes Wort mit einem Ġ startet.
bpe_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

bpe_tokenizer.decoder = ByteLevelDecoder()

corpus = [full_text]

trainer = BpeTrainer(
    vocab_size=10000, # Hier definieren wir wie lange trainiert werden soll.
    min_frequency=1, # Erlaubt es, dass die bytes für Umlaute zusammengefast werden können.
    show_progress=True,
    # Diese speziellen Token bleiben erhalten und kennzeichnen besondere Stellen und
    # Zeichen im Text. Wie etwa unbekannte Zeichen mit [UNK] oder den Start oder
    # Das Ende einer Zeichenkette.
    special_tokens=["[UNK]", "[BOS]", "[EOS]", "[PAD]", "[CLS]", "[SEP]"],
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

bpe_tokenizer.train_from_iterator(
    corpus,
    trainer=trainer
)

print("Tokenizer training complete!")

text_to_encode = "Ich lerne gerne viel über die schöne Welt der KI!"

encoded_text = bpe_tokenizer.encode(text_to_encode).tokens
encoded_ids = bpe_tokenizer.encode(text_to_encode).ids

print(f"\nOriginal text: {text_to_encode}")
print(f"Encoded text (BPE tokens): {encoded_text}")
print(f"Encoded text (BPE IDs): {encoded_ids}")

decoded_text = bpe_tokenizer.decode(encoded_ids)
print(f"Decoded text: {decoded_text}")

Tokenizer training complete!

Original text: Ich lerne gerne viel über die schöne Welt der KI!
Encoded text (BPE tokens): ['ĠIch', 'Ġl', 'er', 'ne', 'Ġgerne', 'Ġviel', 'ĠÃ¼ber', 'Ġdie', 'ĠschÃ¶ne', 'ĠWelt', 'Ġder', 'ĠK', 'I', '!']
Encoded text (BPE IDs): [670, 406, 262, 427, 2703, 993, 465, 292, 4786, 954, 294, 319, 46, 6]
Decoded text:  Ich lerne gerne viel über die schöne Welt der KI!
